# 问句解析

一般问句解析需要进行分词、词性标注、命名实体识别、关键词提取、句法分析以及查询问句分类等。这些事情我们从头开始做无非是重复造轮子，傻子才会这么做，人之所以为人是因为会使用工具。网络上有关中文的NLP工具有很多，介绍几个不错的：

第一个要数哈工大的LTP(语言技术平台)了，它可以做中文分词、词性标注、命名实体识别、依存句法分析、语义角色标注等丰富、 高效、精准的自然语言处理技术

第二个就是博森科技了，它除了做中文分词、词性标注、命名实体识别、依存文法之外还可以做情感分析、关键词提取、新闻分类、语义联想、时间转换、新闻摘要等，但因为是商业化的公司，除了分词和词性标注免费之外全都收费

第三个就是jieba分词，这个开源小工具分词和词性标注做的挺不错的，但是其他方面还欠缺一下，如果只是中文分词的需求完全可以满足

第四个就是中科院张华平博士的NLPIR汉语分词系统，也能支持关键词提取

我们优先选择NLPIR

In [3]:
import sys
import pynlpir

pynlpir.open()
s = '聊天机器人到底该怎么做呢？'
segments = pynlpir.segment(s)
for segment in segments:
    print(segment[0], '\t', segment[1])

pynlpir.close()

聊天 	 verb
机器人 	 noun
到底 	 adverb
该 	 verb
怎么 	 pronoun
做 	 verb
呢 	 modal particle
？ 	 punctuation mark


关键词提取效果：

In [3]:
pynlpir.open()
key_words = pynlpir.get_key_words(s, weighted=True)
for key_word in key_words:
    print(key_word[0],'\t',key_word[1])
pynlpir.close()

聊天 	 2.0
机器人 	 2.0


使用全部的分析功能：

In [12]:
pynlpir.open()
s = '海洋是如何形成的'
segments = pynlpir.segment(s, pos_names='all')
segments

[('海洋', 'noun'),
 ('是', 'verb:verb 是'),
 ('如何', 'pronoun:interrogative pronoun:predicate interrogative pronoun'),
 ('形成', 'verb'),
 ('的', 'particle:particle 的/底')]

In [4]:
pynlpir.open()
segments = pynlpir.segment(s, pos_names='all', pos_english=False)
segments

[('聊天', '动词:不及物动词'),
 ('机器人', '名词'),
 ('到底', '副词'),
 ('该', '动词'),
 ('怎么', '代词:疑问代词:谓词性疑问代词'),
 ('做', '动词'),
 ('呢', '语气词'),
 ('？', '标点符号:问号')]

这里的segment是切词的意思，返回的是tuple(token, pos)，其中token就是切出来的词，pos就是语言属性

调用segment方法指定的pos_names参数可以是'all', 'child', 'parent'，默认是parent， 表示获取该词性的最顶级词性，child表示获取该词性的最具体的信息，all表示获取该词性相关的所有词性信息，相当于从其顶级词性到该词性的一条路径. 

查看nlpir的源代码中的pynlpir/docs/pos_map.rst，可以看出全部词性分类及其子类别如下：
```python
POS_MAP = {
        'n': ('名词', 'noun', {
            'nr': ('人名', 'personal name', {
                'nr1': ('汉语姓氏', 'Chinese surname'),
                'nr2': ('汉语名字', 'Chinese given name'),
                'nrj': ('日语人名', 'Japanese personal name'),
                'nrf': ('音译人名', 'transcribed personal name')
            }),
            'ns': ('地名', 'toponym', {
                'nsf': ('音译地名', 'transcribed toponym'),
            }),
            'nt': ('机构团体名', 'organization/group name'),
            'nz': ('其它专名', 'other proper noun'),
            'nl': ('名词性惯用语', 'noun phrase'),
            'ng': ('名词性语素', 'noun morpheme'),
        }),
        't': ('时间词', 'time word', {
            'tg': ('时间词性语素', 'time morpheme'),
        }),
        's': ('处所词', 'locative word'),
        'f': ('方位词', 'noun of locality'),
        'v': ('动词', 'verb', {
            'vd': ('副动词', 'auxiliary verb'),
            'vn': ('名动词', 'noun-verb'),
            'vshi': ('动词"是"', 'verb 是'),
            'vyou': ('动词"有"', 'verb 有'),
            'vf': ('趋向动词', 'directional verb'),
            'vx': ('行事动词', 'performative verb'),
            'vi': ('不及物动词', 'intransitive verb'),
            'vl': ('动词性惯用语', 'verb phrase'),
            'vg': ('动词性语素', 'verb morpheme'),
        }),
        'a': ('形容词', 'adjective', {
            'ad': ('副形词', 'auxiliary adjective'),
            'an': ('名形词', 'noun-adjective'),
            'ag': ('形容词性语素', 'adjective morpheme'),
            'al': ('形容词性惯用语', 'adjective phrase'),
        }),
        'b': ('区别词', 'distinguishing word', {
            'bl': ('区别词性惯用语', 'distinguishing phrase'),
        }),
        'z': ('状态词', 'status word'),
       'r': ('代词', 'pronoun', {
            'rr': ('人称代词', 'personal pronoun'),
            'rz': ('指示代词', 'demonstrative pronoun', {
                'rzt': ('时间指示代词', 'temporal demonstrative pronoun'),
                'rzs': ('处所指示代词', 'locative demonstrative pronoun'),
                'rzv': ('谓词性指示代词', 'predicate demonstrative pronoun'),
            }),
            'ry': ('疑问代词', 'interrogative pronoun', {
                'ryt': ('时间疑问代词', 'temporal interrogative pronoun'),
                'rys': ('处所疑问代词', 'locative interrogative pronoun'),
                'ryv': ('谓词性疑问代词', 'predicate interrogative pronoun'),
            }),
            'rg': ('代词性语素', 'pronoun morpheme'),
        }),
        'm': ('数词', 'numeral', {
            'mq': ('数量词', 'numeral-plus-classifier compound'),
        }),
        'q': ('量词', 'classifier', {
            'qv': ('动量词', 'verbal classifier'),
            'qt': ('时量词', 'temporal classifier'),
        }),
        'd': ('副词', 'adverb'),
        'p': ('介词', 'preposition', {
            'pba': ('介词“把”', 'preposition 把'),
            'pbei': ('介词“被”', 'preposition 被'),
        }),
        'c': ('连词', 'conjunction', {
            'cc': ('并列连词', 'coordinating conjunction'),
        }),
        'u': ('助词', 'particle', {
            'uzhe': ('着', 'particle 着'),
            'ule': ('了／喽', 'particle 了/喽'),
            'uguo': ('过', 'particle 过'),
            'ude1': ('的／底', 'particle 的/底'),
            'ude2': ('地', 'particle 地'),
            'ude3': ('得', 'particle 得'),
            'usuo': ('所', 'particle 所'),
            'udeng': ('等／等等／云云', 'particle 等/等等/云云'),
            'uyy': ('一样／一般／似的／般', 'particle 一样/一般/似的/般'),
            'udh': ('的话', 'particle 的话'),
            'uls': ('来讲／来说／而言／说来', 'particle 来讲/来说/而言/说来'),
            'uzhi': ('之', 'particle 之'),
            'ulian': ('连', 'particle 连'),
        }),
       'e': ('叹词', 'interjection'),
        'y': ('语气词', 'modal particle'),
        'o': ('拟声词', 'onomatopoeia'),
        'h': ('前缀', 'prefix'),
        'k': ('后缀' 'suffix'),
        'x': ('字符串', 'string', {
            'xe': ('Email字符串', 'email address'),
            'xs': ('微博会话分隔符', 'hashtag'),
            'xm': ('表情符合', 'emoticon'),
            'xu': ('网址URL', 'URL'),
            'xx': ('非语素字', 'non-morpheme character'),
        }),
        'w': ('标点符号', 'punctuation mark', {
            'wkz': ('左括号', 'left parenthesis/bracket'),
            'wky': ('右括号', 'right parenthesis/bracket'),
            'wyz': ('左引号', 'left quotation mark'),
            'wyy': ('右引号', 'right quotation mark'),
            'wj': ('句号', 'period'),
            'ww': ('问号', 'question mark'),
            'wt': ('叹号', 'exclamation mark'),
            'wd': ('逗号', 'comma'),
            'wf': ('分号', 'semicolon'),
            'wn': ('顿号', 'enumeration comma'),
            'wm': ('冒号', 'colon'),
            'ws': ('省略号', 'ellipsis'),
            'wp': ('破折号', 'dash'),
            'wb': ('百分号千分号', 'percent/per mille sign'),
            'wh': ('单位符号', 'unit of measure sign'),
        }),
    }
```

# 语料获取

机器学习需要海量的语料数据，这些数据可以从互联网上直接搜索而来，我们要做的就是利用工具记性关键词提取，然后去互联网上去搜索答案，正如上面我们所做的，如下：

In [6]:
import sys
import pynlpir

pynlpir.open()
s = '怎么才能把电脑里的垃圾文件删除'
key_words = pynlpir.get_key_words(s, weighted=True)
for key_word in key_words:
    print(key_word[0], '\t', key_word[1])
pynlpir.close()

电脑 	 2.0
垃圾 	 2.0
文件 	 2.0
删除 	 1.0


**利用搜索引擎**

有了关键词，想获取预料信息，还需要知道几大搜索引擎的调用接口，首先我们来探索一下百度，百度的接口是这样的：`https://www.baidu.com/s?wd=机器学习 数据挖掘 信息检索` 把wd参数换成我们的关键词就可以拿到相应的结果。

使用 scrapy 进行抓取搜索引擎返回的数据，然后进行解析提取语料。

# 语义依存分析

句法分析是自然语言处理中非常重要的环节，语义依存更是对句子更深层次的分析。

**依存句法**

句法就是句子的法律规则，也就是句子里成分都是按照什么法律规则组织在一起的。而依存句法就是这些成分之间有一种依赖关系。什么是依赖：没有你的话，我存在就是个错误。“北京是中国的首都”，如果没有“首都”，那么“中国的”存在就是个错误，因为“北京是中国的”表达的完全是另外一个意思了。

**语义依存**

“语义”就是说句子的含义，“张三昨天告诉李四一个秘密”，那么语义包括：谁告诉李四秘密的？张三。张三告诉谁一个秘密？李四。张三什么时候告诉的？昨天。张三告诉李四什么？秘密。

**区别**

依存句法强调介词、助词等的划分作用，语义依存注重实词之间的逻辑关系。另外，依存句法随着字面词语变化而不同，语义依存不同字面词语可以表达同一个意思，句法结构不同的句子语义关系可能相同。

---
依存句法分析和语义分析相结合使用，对对方说的话进行依存和语义分析后，一方面可以让计算机理解句子的含义，从而匹配到最合适的回答，另外如果有已经存在的依存、语义分析结果，还可以通过置信度匹配来实现聊天回答。



**依存句法分析**

依存句法分析的基本任务是确定句式的句法结构(短语结构)或句子中词汇之间的依存关系。依存句法分析最重要的两棵树：
- 依存树：子节点依存于父节点
- 依存投射树：实线表示依存联结关系，位置低的成分依存于位置高的成分，虚线为投射线
![img](https://ws4.sinaimg.cn/large/69d4185bly1fygz1l2cojj20h00620v0.jpg)

**依存的公理**
1. 一个句子中只有一个成分是独立的
2. 其他成分直接依存于某一成分
3. 任何一个成分都不能依存于两个或两个以上的成分
4. 如果A成分直接依存于B成分，而C成分在句子中位于A和B之间，那么C或者直接依存于B，或者直接依存于A和B之间的某一成分
5. 中心成分左右两面的其他成分相互不发生关系

什么地方存在依存关系呢？比如合成词（如：国内）、短语（如：英雄联盟）很多地方都是

---
**LTP中的依存关系**

```
主谓关系    SBV           subject-verb           我送她一束花 (我 <-- 送)
动宾关系    VOB直接宾语，   verb-object            我送她一束花 (送 --> 花)
间宾关系    IOB间接宾语，   indirect-object        我送她一束花 (送 --> 她)
前置宾语    FOB前置宾语，   fronting-object        他什么书都读 (书 <-- 读)
兼语       DBL            double                 他请我吃饭 (请 --> 我)
定中关系    ATT           attribute              红苹果 (红 <-- 苹果)
状中结构    ADV           adverbial              非常美丽 (非常 <-- 美丽)
动补结构    CMP           complement             做完了作业 (做 --> 完)
并列关系    COO           coordinate             大山和大海 (大山 --> 大海)
介宾关系    POB           preposition-object     在贸易区内 (在 --> 内)
左附加关系  LAD           left adjunct           大山和大海 (和 <-- 大海)
右附加关系  RAD           right adjunct          孩子们 (孩子 --> 们)
独立结构   IS            independent structure  两个单句在结构上彼此独立
核心关系   HED           head                   指整个句子的核心
```
依存关系是通过机器学习和人工标注来完成的，机器学习依赖人工标注，那么都哪些需要我们做人工标注呢？分词词性、依存树库、语义角色都需要做人工标注，有了这些人工标注之后，就可以做机器学习来分析新的句子的依存句法了。

# 语言模型

无论什么做自然语言处理的工具，都是基于计算机程序实现的，而计算机承担了数学计算的职责，那么自然语言和数学之间的联系就是语言模型，只有理解语言模型才能理解各种工具的实现原理。

**数学模型**

数学模型是运用数理逻辑方法和数学语言建构的科学或工程模型。说白了，就是用数学的方式来解释事实。

**数学建模**

数学建模就是通过计算得到的结果来解释实际问题，并接受实际的检验，来建立数学模型的全过程。

**语言模型**

语言模型是根据语言客观事实而进行的语言抽象数学建模。说白了，就是找到一个数学模型，让它来解释自然语言的事实。

**基于概率的统计**

说到基于统计，那么就要说概率是如何估计的了，通常都是使用最大似然估计，怎么样理解“最大似然估计”，最大似然就是最最最最最相似的，那么和谁相似，和历史相似，历史是什么样的？10个词里出现过2次，所以是2/10=1/5，所以经常听说过的“最大似然估计”就是用历史出现的频率来估计概率的方法。

# 中文分词

中文自动分词最难的两个问题：1）歧义消除；2）未登陆词识别。 未登录词就是新词，词表里没有的词。

几种统计分词方法：N-最短路径分词法、基于n元语法模型的分词法：
- N-最短路径分词法其实就是一元语法模型，每个词成为一元，独立存在，出现的概率可以基于大量语料统计得出，比如“确实”这个词出现概率的0.001，我们把一句话基于词表的各种切词结果都列出来，因为字字组合可能有很多种，所以有多个候选结果，这时我们利用每个词出现的概率相乘起来，得到的最终结果，谁最大谁就最有可能是正确的，这就是N-最短路径分词法。

这里的N的意思是说我们计算概率的时候最多只考虑前N个词，因为一个句子可能很长很长，词离得远，相关性就没有那么强了。这里的最短路径其实是传统最短路径的一种延伸，由加权延伸到了概率乘积。

- 基于n元语法模型的分词法就是在N-最短路径分词法基础上把一元模型扩展成n元模型，也就是统计出的概率不再是一个词的概率，而是基于前面n个词的条件概率

# 概率图模型

- [自己动手做聊天机器人 十五-一篇文章读懂拿了图灵奖和诺贝尔奖的概率图模型](http://www.shareditor.com/blogshow?blogId=81)

**贝叶斯网络**

提到贝叶斯就是条件概率，所以也就是生成式模型，也就是有向图模型，如下图：
![img](https://wx3.sinaimg.cn/large/69d4185bly1fyhmsakc99j206k07q0t3.jpg)

图中每一个点都可能为True或False，他们的概率是已知的，比如x7的概率需要有x4和x5来决定，可能是这样的：
```
x4 x5   T    F

T  T   0.5 0.5

T  F   0.4 0.6

F  T   0.7 0.3

F  F   0.2 0.8
```

那么可以通过上面的贝叶斯网络来估计如果x1为False情况下x6为True的概率：

P(x6=T|x1=F)=P(x6=T,x1=F)/P(x1=F)

这个值继续推导，最终可以由每个节点的概率数据计算求得，这么说来，贝叶斯网络模型可以通过样本学习来估计每个节点的概率，从而达到可以预测各种问题的结果。

贝叶斯网络能够在已知有限的、不完整的、不确定信息条件下进行学习推理，所以广泛应用在故障诊断、维修决策、汉语自动分词、词义消歧等问题上。

**马尔科夫模型**

提到马尔可夫就是一个值跟前面n个值有关，所以也就是条件概率，也就是生成式模型，也就是有向图模型：
![img](https://ws2.sinaimg.cn/large/69d4185bly1fyhn1c88ghj20cj09cgnk.jpg)

音乐的每一个音不是随意作出来的，是根据曲子的风格、和弦、大小调式等来决定的，但是因为可选的音高有多种，也就出现了无数美妙的旋律。因为有约束，所以其实可以说新的音和前面的n个音有关，这其实是一个马尔可夫模型可以解释的事情。

马尔可夫模型还可以看成是一个关于时间t的状态转换过程，也就是随机的有限状态机，那么状态序列的概率可以通过计算形成该序列所有状态之间转移弧上的概率乘积得出。如果说这个马尔可夫是两阶的，那么转移概率可能是这个样子：
![img](https://wx1.sinaimg.cn/large/69d4185bly1fyhn2nimh0j20au074tck.jpg)

这种情况由前两列决定的第三列任意值都会有一个概率。我们通过训练样本来得出每一个概率值，这样就可以通过训练出的模型来根据前两个音是什么而预测下一个音是1、2、3、4、5任意一个的概率是多少了，也就是可以自动作曲了，当然这样做出的曲子肯定是一个无线循环的旋律。

那么我们再说隐马尔可夫模型，这里的“隐”指的是其中某一阶的信息我们不知道，就像是我们知道人的祖先是三叶虫，但是由三叶虫经历了怎样的演变过程才演变到人的样子我们是不知道的，我们只能通过化石资料了解分布信息，如果这类资料很多，那么就可以利用隐马尔可夫模型来建模，因为缺少的信息较多，所以这一模型的算法比较复杂，比如前向算法、后向算法之类晦涩的东西就不说了。相对于原理，我们更关注它的应用，隐马尔可夫模型广泛应用在词性标注、中文分词等，为什么能用在这两个应用上呢？仔细想一下能看得出来，比如中文分词，最初你是不知道怎么分词的，前面的词分出来了，你才知道后面的边界在哪里，但是当你后面做了分词之后还要验证前面的分词是否正确，这样前后有依赖关系，而不确定中间状态的情况最适合用隐马尔可夫模型来解释。

**最大熵**

看到熵那么一定会用到 $H(p)=-∑p\log p $，怎么理解最大熵模型呢？我们的最终目的是想知道在某一个信息条件 B 下，得出某种可能的结果 A 的最大的概率，也就是条件概率 $P(A|B)$ 最大的候选结果。因为*最大熵就是不确定性最大，其实也就是条件概率最大，所以求最大的条件概率等同于求最大熵*，而我们这里的熵其实是$H(p)=H(A|B)=-∑p(b)p(a|b)\log(p(a|b))$，为了使用训练数据做估计，这里的 $p(a|b )$ 可以通过训练数据的某些特征来估计，比如这些特征是 $f(a,b)$ ，那么做模型训练的过程就编程了训练 $∑λf(a,b)$ 中的 λ 参数的过程，至此就有些像机器学习的线性回归了，该怎么做就清晰了。所以其实最大熵模型就是利用熵的原理和熵的公式来用另外一种形式来描述具有概率规律的现实的。

**条件随机场**
场表示取值范围，随机场表示随机变量有取值范围，也就是每个随机变量有固定的取值，条件指的是随机变量的取值由一定的条件概率决定，而这里的条件来自于我们有一些观察值，这是它区别于其他随机场的地方。条件随机场也可以看做是一个无向图模型，它特殊就特殊在给定观察序列X时某个特定的标记序列Y的概率是一个指数函数 $exp(∑λt+∑μs)$，其中 t 是转移函数， s 是状态函数，我们需要训练的是 $λ$ 和 $μ$。条件随机场主要应用在标注和切分有序数据上，尤其在自然语言处理、生物信息学、机器视觉、网络智能等方面

---
总结一下，概率图模型包括多种结合概率论和图论的模型，根据特定场景特定需求选择不同的模型，每种模型的参数都需要大量样本训练得出，每种模型都是用来根据训练出来的概率做最优结论选择的，比如根据训练出来的模型对句子做最正确的词性标注、实体标注、分词序列等。

# 自然语言处理

**命名实体**

命名实体包括：人名(政治家、艺人等)、地名(城市、州、国家、建筑等)、组织机构名、时间、数字、专有名词(电影名、书名、项目名、电话号码等)、……。其实领域很多，不同人需求不一样，关注的范围也不一样。总之不外乎命名性指称、名词性指称和代词性指称。

**命名实体的放射性**

为什么说命名实体是有放射性的呢？举个栗子：“中国积极参与亚太经合组织的活动”，这里面的“亚太经合组织”是一个命名实体，定睛一瞧，这个实体着实不凡啊，有“组织”两个字，这么说来这个实体是一种组织或机构，记住，下一次当你看到“组织”的时候和前面几个字组成的一定是一个命名实体。继续观察，在它之前辐射出了“参与”一次，经过大规模语料训练后能发现，才“参与”后面有较大概率跟着一个命名实体。继续观察，在它之后有“的活动”，那么说明前面很可能是一个组织者，组织者多半是一个命名实体。这就是基于条件随机场做命名实体识别的奥秘，这就是命名实体的放射性。

**特征模板**

特征模板的选择是和具体我们要识别的实体类别有关系的，识别人名和识别机构名用的特征模板是不一样的，因为他们的特点就不一样，事实上识别中文人名和识别英文人名用的特征模板也是不一样的，因为他们的特点就不一样。

# 词性标注方法

**词性**

常说的词性包括：名、动、形、数、量、代、副、介、连、助、叹、拟声。但自然语言处理中要分辨的词性要更多更精细，比如：区别词、方位词、成语、习用语、机构团体、时间词等，多达100多种。

汉语词性标注最大的困难是“兼类”，也就是一个词在不同语境中有不同的词性，而且很难从形式上识别。

**标注过程和方法**

为了解决词性标注无法达到100%准确的问题，词性标注一般要经过“标注”和“校验”两个过程，第一步“标注”根据规则或统计的方法做词性标注，第二步“校验”通过一致性检查和自动校对等方法来修正。

词性标注具体方法包括：基于统计模型的方法、基于规则的方法和两者结合的方法。

**统计模型方法**

提到基于统计模型，势必意味着我们要利用大量已经标注好的语料库来做训练，同时要先选择一个合适的训练用的数学模型。

模型参数初始化是在我们尚未利用语料库之前用最小的成本和最接近最优解的目标来设定初值。HMM是一种基于条件概率的生成式模型，所以模型参数是生成概率，那么我们不妨就假设每个词的生成概率就是它所有可能的词性个数的倒数，这个是计算最简单又最有可能接近最优解的生成概率了。每个词的所有可能的词性是我们已经有的词表里标记好的，这个词表的生成方法就比较简单了，我们不是有已经标注好的语料库嘛，很好统计。那么如果某个词在词表里没有呢？这时我们可以把它的生成概率初值设置为0。这就是隐马尔可夫模型参数初始化的技巧，总之原则就是用最小的成本和最接近最优解的目标来设定初值。一旦完成初始值设定后就可以利用前向后向算法进行训练了。

**基于规则的方法**

规则就是我们既定好一批搭配关系和上下文语境的规则，判断实际语境符合哪一种则按照规则来标注词性。这种方法比较古老，适合于既有规则，对于兼词的词性识别效果较好，但不适合于如今网络新词层出不穷、网络用语新规则的情况。

**校验**

第一种校验方法就是检查词性标注的一致性。一致性指的是在所有标注的结果中，具有相同语境下同一个词的标注是否都相同，那么是什么原因导致的这种不一致呢？一种情况就是这类词就是兼类词，可能被标记为不同词性。另一种情况是非兼类词，但是由于人工校验或者其他原因导致标记为不同词性。达到100%的一致性是不可能的，所以我们需要保证一致性处于某个范围内，由于词数目较多，词性较多，一致性指标无法通过某一种计算公式来求得，因此可以基于聚类和分类的方法，根据欧式距离来定义一致性指标，并设定一个阈值，保证一致性在阈值范围内。

第二种校验方法就是词性标注的自动校对。自动校对顾名思义就是不需要人参与，直接找出错误的标注并修正，这种方法更适用于一个词的词性标注通篇全错的情况，因为这种情况基于数据挖掘和规则学习方法来做判断会相对比较准确。通过大规模训练语料来生成词性校对决策表，然后根据这个决策表来找通篇全错的词性标注并做自动修正。

---
词性标注的方法主要有基于统计和基于规则的方法，另外还包括后期校验的过程。词性标注是帮助计算机理解语言含义的关键，有了词性标注，我们才可以进一步确定句法和语义，才有可能让机器理解语言的含义，才有可能实现聊天机器人的梦想

# 句法分析树

句法分析分为句法结构分析和依存关系分析.

句法结构分析也就是短语结构分析，比如提取出句子中的名次短语、动词短语等，最关键的是人可以通过经验来判断的短语结构.

**基本方法**

分为基于规则的分析方法和基于统计的分析方法。基于规则的方法存在很多局限性，所以我们采取基于统计的方法，目前最成功的是基于概率上下文无关文法(PCFG)。基于PCFG分析需要有如下几个要素：终结符集合、非终结符集合、规则集。

我们的终结符集合是：∑={我, 吃, 肉,……}，这个集合表示这三个字可以作为句法分析树的叶子节点，当然这个集合里还有很多很多的词。

我们的非终结符集合是：N={S, VP, ……}，这个集合表示树的非页子节点，也就是连接多个节点表达某种关系的节点，这个集合里也是有很多元素。

我们的规则集：
```
R={
NN->我    0.5
Vt->吃     1.0
NN->肉   0.5
VP->Vt NN    1.0
S->NN VP 1.0
……
}
```

后面一列是模型训练出来的概率值，也就是在一个固定句法规则中NN的位置是“我”的概率是0.5，NN推出“肉”的概率是0.5，0.5+0.5=1，也就是左部相同的概率和一定是1 。

再换一种方法解释一下，有一种句法规则是：
```
S——|
|     |
NN    VP
     |——|
  Vt     NN
```
其中NN的位置可能是“我”，也可能是“肉”，是“我”的概率是0.5，是“肉”的概率是0.5，两个概率和必为1。其中Vt的位置一定是“吃”，也就是概率是1.0 。

那么如何根据以上的几个要素来生成句法分析树呢？  
（1）“我”：词性是NN，推导概率是0.5，树的路径是“我”  
（2）“吃”：词性是Vt，推导概率是1.0，树的路径是“吃”  
（3）“肉”：词性是NN，概率是0.5，和Vt组合符合VP规则，推导概率是0.5\*1.0\*1.0=0.5，树的路径是“吃肉”

NN和VP组合符合S规则，推导概率是0.5\*0.5\*1.0=0.25，树的路径是“我吃肉”

所以最终的树结构是：
```
S——|
|      |
NN     VP
我    |——|
    Vt     NN
    吃     肉
```

上面的计算过程总结起来就是：设 $W={ω_1,ω_2,ω_3……}$ 表示一个句子，其中的 $ω$ 表示一个词(word)，利用动态规划算法计算非终结符 A推 导出 W 中子串 $ω_iω_{i+1}ω_{i+2}……ω_j$ 的概率，假设概率为 $α_{ij}(A)$，那么有如下递归公式：
$$
\begin{split}
α_{ij}(A) &= P(A \rightarrow ω_i) \\
α_{ij}(A) &= ∑∑P(A \rightarrow BC)α_{ik}(B)α_{(k+1)j}(C)
\end{split}
$$
以上两个式子好好理解一下其实就是上面“我吃肉”的计算过程

**句法规则提取**

首先我们需要大量的树库，也就是训练数据。然后我们把树库中的句法规则提取出来生成我们想要的结构形式，并进行合并、归纳等处理，最终得到上面∑、N、R的样子。其中的概率参数计算方法是这样的：

先给定参数为一个随机初始值，然后采用EM迭代算法，不断训练数据，并计算每条规则使用次数作为最大似然计算得到概率的估值，这样不断迭代更新概率，最终得出的概率可以认为是符合最大似然估计的精确值。

---
句法分析树生成算法是基于统计学习的原理，根据大量标注的语料库（树库），通过机器学习算法得出非终结符、终结符、规则集及其概率参数，然后利用动态规划算法生成每一句话的句法分析树，在句法分析树生成过程中如果遇到多种树结构，选择概率最大的那一种作为最佳句子结构。

# 语义分析

**词义消歧**

词义消歧是句子和篇章语义理解的基础，是必须解决的问题。任何一种语言都有大量具有多种含义的词汇，中文的“日”，英文的“bank”，法语的“prendre”……。

词义消歧可以通过机器学习的方法来解决。谈到机器学习就会分成有监督和无监督的机器学习。词义消歧有监督的机器学习方法也就是分类算法，即判断词义所属的分类。词义消歧无监督的机器学习方法也就是聚类算法，把词义聚成多类，每一类是一种含义。

**有监督 - 基于互信息**

用两种语言对照着看，比如：中文“打人”对应英文“beat a man”，而中文“打酱油”对应英文“buy some sauce”。这样就知道当上下文语境里有“人”的时候“打”的含义是beat，当上下文语境里有“酱油”的时候“打”的含义是buy。按照这种思路，基于大量中英文对照的语料库训练出来的模型就可以用来做词义消歧了，这种方法就叫做基于“互信息”的词义消歧方法。

互信息，它来源于信息论，表达的是一个随机变量中包含另一个随机变量的信息量(也就是英文信息中包含中文信息的信息量)，假设两个随机变量 X、Y 的概率分别是 p(x), p(y)，它们的联合分布概率是 p(x,y)，那么互信息计算公式是：
$$
I(X; Y) = ∑∑ p(x,y)\log\left(\frac{p(x,y)}{p(x)p(y)}\right)
$$

“互信息”可以理解为一个随机变量由于已知另一个随机变量而减少的不确定性(也就是理解中文时由于已知了英文的含义而让中文理解更确定了)，因为“不确定性”就是熵所表达的含义，所以：
$$
I(X; Y) = H(X) - H(X|Y)
$$

那么我们在对语料不断迭代训练过程中I(X; Y)是不断减小的，算法终止的条件就是I(X; Y)不再减小。基于互信息的词义消歧方法自然对机器翻译系统的效果是最好的，但它的缺点是：*双语语料有限*，多种语言能识别出歧义的情况也是有限的(比如中英文同一个词都有歧义就不行了)。

**有监督 - 基于贝叶斯分类器**

提到贝叶斯那么一定少不了条件概率，这里的条件指的就是上下文语境这个条件，任何多义词的含义都是跟上下文语境相关的。假设语境 (context) 记作 c，语义(semantic)记作 s，多义词(word)记作 w，那么我要计算的就是多义词 w 在语境 c下具有语义 s 的概率: $p(s|c)$

那么根据贝叶斯公式：$p(s|c) = p(c|s)p(s)/p(c)$，我要计算的就是p(s|c)中s取某一个语义的最大概率，因为p(c)是既定的，所以只考虑分子的最大值：$s的估计=max(p(c|s)p(s))$。

因为语境c在自然语言处理中必须通过词来表达，也就是由多个 v (词)组成，那么也就是计算：$max(p(s)∏p(v|s))$.

下面就是训练的过程了： p(s)表达的是多义词 w 的某个语义 s 的概率，可以统计大量语料通过最大似然估计求得：$p(s) = N(s)/N(w)$。p(v|s)表达的是多义词w的某个语义s的条件下出现词v的概率，可以统计大量语料通过最大似然估计求得：$p(v|s) = N(v, s)/N(s)$ 。训练出p(s)和p(v|s)之后我们对一个多义词w消歧的过程就是计算 p(c|s)p(s) 的最大概率的过程。

**无监督**

完全无监督的词义消歧是不可能的，因为没有标注是无法定义是什么词义的，但是可以通过无监督的方法来做词义辨识。无监督的词义辨识其实也是一种贝叶斯分类器，和上面讲到的贝叶斯分类器消歧方法不同在于：这里的参数估计不是基于有标注的训练预料，而是先随机初始化参数p(v|s)，然后根据EM算法重新估计这个概率值，也就是对w的每一个上下文c计算p(c|s)，这样可以得到真实数据的似然值，回过来再重新估计p(v|s)，重新计算似然值，这样不断迭代不断更新模型参数，最终得到分类模型，可以对词进行分类，那么有歧义的词在不同语境中会被分到不同的类别里。

# 语义角色标注

举个栗子：“我昨天吃了一块肉”，按照常规理解“我吃肉”应该是句子的核心，但是对于机器来说“我吃肉”实际上已经丢失了非常多的重要信息，没有了时间，没有了数量。为了让机器记录并提取出这些重要信息，句子的核心并不是“我吃肉”，而是以谓词“吃”为核心的全部信息。

“吃”是谓词，“我”是施事者，“肉”是受事者，“昨天”是事情发生的时间，“一块”是数量。语义角色标注就是要分析出这一些角色信息，从而可以让计算机提取出重要的结构化信息，来“理解”语言的含义。

语义角色标注需要依赖句法分析的结果进行，因为句法分析包括短语结构分析、浅层句法分析、依存关系分析，所以语义角色标注也分为：基于短语结构树的语义角色标注方法、基于浅层句法分析结果的语义角色标注方法、基于依存句法分析结果的语义角色标注方法。但无论哪种方法，过程都是：

句法分析->候选论元剪除->论元识别->论元标注->语义角色标注结果：
- 其中论元剪除就是在较多候选项中去掉肯定不是论元的部分
- 其中论元识别是一个二值分类问题，即：是论元和不是论元
- 其中论元标注是一个多值分类问题